In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.metrics import f1_score, accuracy_score

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive//My Drive/Andmeteadus/Masinõpe/4N/")

Mounted at /content/drive


In [3]:
data = pd.read_csv("cleaned_with_title_am_nam.csv", sep="\t")
print(data.shape)
data.dropna()
print(data.shape)
w2v = pd.read_csv("word2vec_features_with_labels_after_removing_noisy_columns.csv")

(1218, 4)
(1218, 4)


In [4]:
#!wget -O w2v.zip https://owncloud.ut.ee/owncloud/index.php/s/3GbKqZi3pHGsDHD/download
#!unzip w2v.zip
w2vmodel = gensim.models.KeyedVectors.load_word2vec_format("model.bin", binary=True, unicode_errors='replace')

In [5]:
train, test = train_test_split(data, train_size=0.8, test_size=0.2, shuffle=True)
def preprocess(data):
  new_articles = []
  for i in data.iterrows():
    if type(i[1]["title"]) == float or type(i[1]["article"]) == float:
      new_articles.append("")
      continue

    new_articles.append(i[1]["title"]+" "+i[1]["article"])
  data["data"] = np.array(new_articles)
  return data
#data.head()

In [6]:
le = LabelEncoder()
train = preprocess(train)
train_X = train["data"]
train_y = train["label"]
train_y = to_categorical(le.fit_transform(train_y))

test = preprocess(test)
test_X = test["data"]
test_y = test["label"]
test_y = to_categorical(le.transform(test_y))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [7]:
test_y.shape, test_X.shape, train_X.shape, train_y.shape

((244, 2), (244,), (974,), (974, 2))

In [8]:
vocabulary_size = 10000
tokenizer = Tokenizer(num_words=vocabulary_size,oov_token="XXX") 
tokenizer.fit_on_texts(train_X)
X_train_vec = tokenizer.texts_to_sequences(train_X)
X_test_vec = tokenizer.texts_to_sequences(test_X)
pikkus = 1000
X_train_pad = sequence.pad_sequences(X_train_vec, maxlen=pikkus)
X_test_pad = sequence.pad_sequences(X_test_vec, maxlen=pikkus)
y_train_vec = train_y
y_test_vec = test_y
checked_vocabulary_size = min(vocabulary_size, len(tokenizer.word_index)+1)
embedding_size = 300
embedding_matrix = np.zeros((checked_vocabulary_size, embedding_size))
for word, i in tokenizer.word_index.items():
  if i>=vocabulary_size:
    continue
  try:
    embedding_matrix[i] = w2vmodel[word]
  except KeyError:
    embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_size)

model = Sequential()
model.add(Embedding(checked_vocabulary_size,embedding_size,weights=[embedding_matrix],trainable=True))
model.add(LSTM(35, return_sequences=True, recurrent_dropout=0.2, dropout=0.1))
model.add(LSTM(35, return_sequences=False, recurrent_dropout=0.2, dropout=0.1))
model.add(Dense(10, activation="relu"))
model.add(Dense(2, activation='softmax'))

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 300)         3000000   
_________________________________________________________________
lstm (LSTM)                  (None, None, 35)          47040     
_________________________________________________________________
lstm_1 (LSTM)                (None, 35)                9940      
_________________________________________________________________
dense (Dense)                (None, 10)                360       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
Total params: 3,057,362
Trainable params: 3,057,362
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(X_train_pad, y_train_vec,
          batch_size=64,
          epochs=4,
          validation_data=(X_test_pad, y_test_vec))

Epoch 1/4
16/16 [==============================] - 62s 4s/step - loss: 0.6786 - accuracy: 0.5657 - val_loss: 0.6514 - val_accuracy: 0.5861
Epoch 2/4
16/16 [==============================] - 63s 4s/step - loss: 0.5831 - accuracy: 0.7033 - val_loss: 0.5900 - val_accuracy: 0.7008
Epoch 3/4
16/16 [==============================] - 64s 4s/step - loss: 0.4291 - accuracy: 0.8326 - val_loss: 0.5507 - val_accuracy: 0.7295
Epoch 4/4
16/16 [==============================] - 63s 4s/step - loss: 0.2525 - accuracy: 0.9097 - val_loss: 0.6439 - val_accuracy: 0.7336


In [13]:
y_pred_vec = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_vec, axis=1)
y_true = np.argmax(y_test_vec, axis=1)

f1_score(y_true, y_pred), accuracy_score(y_true, y_pred)

(0.6666666666666666, 0.7336065573770492)